In [1]:
# Data preprocessing, math and plotting
import os
import numpy as np 
import pandas as pd 
from scipy import stats 
import matplotlib.pyplot as plt
import seaborn as sns 
# ML 
import sklearn
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.metrics import r2_score, confusion_matrix

In [2]:
customers_df = pd.read_csv(r"data\olist_customers_dataset.csv")
geolocation_df = pd.read_csv(r'data/olist_geolocation_dataset.csv') # drop
orders_df = pd.read_csv(r'data/olist_orders_dataset.csv')
order_items_df = pd.read_csv(r'data/olist_order_items_dataset.csv')
order_payments_df = pd.read_csv(r'data/olist_order_payments_dataset.csv')
reviews_df = pd.read_csv(r'data/olist_order_reviews_dataset.csv') 
products_df = pd.read_csv(r'data/olist_products_dataset.csv')
sellers_df = pd.read_csv(r'data/olist_sellers_dataset.csv') 
category_names_translated_df = pd.read_csv(r'data/product_category_name_translation.csv') 
# Get product category translation
products_translated_df = category_names_translated_df.merge(products_df, on='product_category_name', how='left')
products_translated_df.drop(products_translated_df.columns[[0]], axis=1, inplace=True)

In [3]:
merged_df = orders_df.merge(order_items_df, on='order_id', how='left')
merged_df = merged_df.merge(order_payments_df, on='order_id', how='outer', validate='m:m')
merged_df = merged_df.merge(reviews_df, on='order_id', how='outer')
merged_df = merged_df.merge(customers_df, on='customer_id', how='outer')
merged_df = merged_df.merge(sellers_df, on='seller_id', how='outer')
merged_df = merged_df.merge(products_translated_df, on='product_id', how='outer')

### Create Model-based Collaborative filtering system for Product Recommendation

##### Recommend items to customers based on the purchase history and similarity of ratings provided by other customers. Items bought by the same customers, with the highest ratings, will be recommended to other customers with similar purchase history, weighted by their item rankings. Using a collaborative filtering technique helps predict products that a customer might buy, based on the patterns seen by customer specific preferences.

##### Utility Matrix & Singular Value Decomposition (SVD)
###### A utility matrix consits of all possible customer-item preferences, using item similarity weighted by the customer's ratings of said items. The data associated with each customer represents how much the customer appreciates the item, in respect to others who purchase the same item.
###### The SVD transformer performs linear dimensionality reduction by means of truncated singular value decomposition (SVD). Unlike PCA, principle component analysis, this estimator does not center the data before computing the singular value decomposition. This means that it can efficiently work with sparse matices.

In [10]:
# subset of dataset 'df_train'
df_train_subset = merged_df.head(10000)

df_train_util_matrix = df_train_subset.pivot_table(values='review_score', index='customer_id', columns='product_id', fill_value=0)
df_train_util_matrix.head()
            
# transpose the matrix
X = df_train_util_matrix.T
X.head()

# unique products in subset of data
X1 = X

# SVD transformation
SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
print(decomposed_matrix.shape)

# Correlation Matrix
correlation_matrix = np.corrcoef(decomposed_matrix)

# Isolating Product ID # 009c09f439988bc06a93d6b8186dce73 from the Correlation Matrix (random selection based on index value[1])
product = X.index[1]
print(product)

# Index number and product ID purchased by customer 
product_names = list(X.index)
product_id = product_names.index(product)

# Correlation for all items with the item purchased by this customer based on items rated by other customers people who bought the same product
correlation_product_ID = correlation_matrix[product_id]
correlation_product_ID.shape

# Recommending the most highly correlated products in sequence with scores of over 0.90
Recommend = list(X.index[correlation_product_ID > 0.90])

# Removes the item already bought by the customer 
Recommend.remove(i) 

(1198, 10)
009c09f439988bc06a93d6b8186dce73


#### Below are the top products to be displayed by the recommendation system to the above customer based on the purchase history of other customers in the website (Correlation score over 90%)
#### (only if we knew what each product ID represented....)

In [6]:
Recommend

['0faf79119af5cb143f82d98d27d9c5e3',
 '1e96993bf0abebd3ee50f96a52bb51aa',
 '1ede3c17889b7e18f4ac33d2fd0bef96',
 '2362291929f77eccf5f5cb26029dcf90',
 '3225c54bb2785c33f49f50398fcdb88c',
 '3386491e65bf02586461d30dc3fe53f1',
 '35b1777d7b1b0e7839a9d0bd138f4c76',
 '361b81e3cae3b21bd0119793c5399717',
 '3bf87d2001b8176e6a3ad804b80774b4',
 '4107fe37acd2074de27143fa85966f67',
 '441d07171060d7bca33d9946d792da25',
 '63b2c54ffb7ded0576c3bd1346a53d1c',
 '6a33f85f66191cc5b645b5f775ca93d9',
 '6d6c510f2222e168bf3b6b67e24c9535',
 '7c45216f5d973dbd72f3a6e5186438c0',
 '8fd57e95e8bf639d562620356071bc23',
 '986af931e7ce399914deaa43d224b124',
 '99e6f14f74a00605ad9cfe8c4bb29385',
 'b7c8f19a99c322aa3d90d93fbe9eb275',
 'cad39e71c0fe2f6ce875040d4ac43a4b',
 'ce846b23f06e1c42afbc68efc1d35b82',
 'd4bf1e29f41deda5c71da49e4d39fb1d',
 'd9bdf643d95cb89844c0da1a0df1d16e',
 'da97969972a524bca894f97ea6bcff90',
 'ddf035e3aa3fdf3d2869e8be378d001a',
 'e5196a46c6c0e0874d6e41ed149dcf20',
 'e99d30bfed97b6061153d250f19d381b']